<a href="https://colab.research.google.com/github/denisangelo/Federated_Learning/blob/main/Aula_FL_com_Divisao_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [233]:
#Importanto Pytorch, Flower,Opacus
!pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [234]:
from collections import OrderedDict
from typing import List, Tuple
import matplotlib.pyplot as plt

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from flwr.common import Metrics
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [235]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using Pytorch{torch.__version__}, Flower{fl.__version__}" )

Training on cpu using Pytorch2.0.1+cu118, Flower1.4.0


In [236]:
BATCH_SIZE = 32
NUM_CLIENTS= 2
optim_lr=0.001
local_epochs=1
num_rounds=5

In [237]:
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10(root="./data", train=True,download=True, transform=transform)
    testset = CIFAR10(root="./data", train=False, download=True, transform=transform)
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10 # 10% do conjunto para testes
        len_train = len(ds) - len_val
        lengths =[len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,num_workers=0))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False,num_workers=0))
        testloader = DataLoader(testset,batch_size=BATCH_SIZE, shuffle=False,num_workers=0)
        num_examples = {"ds_train": len(ds_train)*BATCH_SIZE, "ds_val": len(ds_val)*BATCH_SIZE}

    return trainloaders, valloaders, testloader,num_examples
    
trainloaders, valloaders, testloader,num_examples = load_data()

Files already downloaded and verified
Files already downloaded and verified


In [238]:
# Definindo o Modelo
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [239]:
# Definindo as funções de treinamento
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [240]:
# Funções de atualização de parâmetros do modelo
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [242]:
# Definindo a classe de cliente Virtual
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader, num_example)-> None:
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.num_example = num_example

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
        net = Net().to(DEVICE)
        trainloader = trainloaders[int(cid)]
        valloader = valloaders[int(cid)]
        num_example = num_examples
        return FlowerClient(cid, net, trainloader, valloader, num_example)

In [243]:
# Realizando a agregação
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [244]:
# Iniciando o treino federado
# Criando a Função de agregação FedAvg
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,#-->Fração de clientes necessários para treinamento
    fraction_evaluate=1.0, #--> Fração de clientes que serão utilizados para teste
    min_fit_clients=NUM_CLIENTS, #--> Número mínimo que serão utlilizados para treinamento
    min_evaluate_clients=2, #--> Número mínimo de clientes que serão realizados para teste
    min_available_clients=NUM_CLIENTS,#--> Número mínimo de clientes que serão utilizados no treinamento
    evaluate_metrics_aggregation_fn=weighted_average,  
)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Inicia a simulação
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-08 13:43:19,746 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-06-08 13:43:25,819	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-06-08 13:43:29,274 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 3903472435.0, 'memory': 7806944871.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 3903472435.0, 'memory': 7806944871.0, 'CPU': 2.0}
INFO flwr 2023-06-08 13:43:29,287 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-08 13:43:29,295 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=31777) 2023-06-08 13:43:32.762454: W tensorflow/compiler/

(launch_and_get_parameters pid=31777) [Client 1] get_parameters
(launch_and_fit pid=31777) [Client 0] fit, config: {}


(pid=31778) 2023-06-08 13:43:46.258966: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_fit pid=31778) [Client 1] fit, config: {}


DEBUG flwr 2023-06-08 13:44:19,393 | server.py:232 | fit_round 1 received 2 results and 0 failures
DEBUG:flwr:fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-06-08 13:44:19,423 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-06-08 13:44:19,434 | server.py:168 | evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG:flwr:evaluate_round 1: strategy sampled 2 clients (out of 2)


...

(launch_and_fit pid=31778) [Client 1] fit, config: {}
(launch_and_evaluate pid=31778) [Client 1] fit, config: {}


DEBUG flwr 2023-06-08 13:46:57,892 | server.py:232 | fit_round 5 received 2 results and 0 failures
DEBUG:flwr:fit_round 5 received 2 results and 0 failures
DEBUG flwr 2023-06-08 13:46:57,905 | server.py:168 | evaluate_round 5: strategy sampled 2 clients (out of 2)
DEBUG:flwr:evaluate_round 5: strategy sampled 2 clients (out of 2)


(launch_and_evaluate pid=31777) [Client 1] evaluate, config: {}
(launch_and_fit pid=31777) [Client 1] evaluate, config: {}


DEBUG flwr 2023-06-08 13:47:03,389 | server.py:182 | evaluate_round 5 received 2 results and 0 failures
DEBUG:flwr:evaluate_round 5 received 2 results and 0 failures
INFO flwr 2023-06-08 13:47:03,397 | server.py:147 | FL finished in 206.65226078399974
INFO:flwr:FL finished in 206.65226078399974
INFO flwr 2023-06-08 13:47:03,404 | app.py:218 | app_fit: losses_distributed [(1, 0.05305677287578582), (2, 0.04399799003601074), (3, 0.04091858224868774), (4, 0.03825138065814972), (5, 0.03701297923326492)]
INFO:flwr:app_fit: losses_distributed [(1, 0.05305677287578582), (2, 0.04399799003601074), (3, 0.04091858224868774), (4, 0.03825138065814972), (5, 0.03701297923326492)]
INFO flwr 2023-06-08 13:47:03,406 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-08 13:47:03,412 | app.py:220 | app_fit: metrics_distributed {'accuracy': [(1, 0.4094), (2, 0.4944), (3, 0.532), (4, 0.5668), (5, 0.5738)]}
INFO:flwr:app_fit: metrics_distributed 

History (loss, distributed):
	round 1: 0.05305677287578582
	round 2: 0.04399799003601074
	round 3: 0.04091858224868774
	round 4: 0.03825138065814972
	round 5: 0.03701297923326492
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.4094), (2, 0.4944), (3, 0.532), (4, 0.5668), (5, 0.5738)]}